# Spoon Gift Checker


## ライブラリのインポート

In [1]:
from tempfile import mkdtemp

import os
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException

## 設定パラメータ

In [2]:
# 取得したい枠のURL
live_url = "https://www.spooncast.net/jp/live/@sirokumapapa"

# コメント読み取り頻度(秒)
# 小さくするとSpoonサーバーに負荷がかかるため、
# 最低でも1sec程度に留めておく
wait_time = 1

# コメントのバッファ
# wait_time秒の間で更新できる最新コメントの数
# PC性能やネットワーク速度が良いほど、上限を上げることができる。
# 上げすぎるとスクレイピングの速度が足りずに挙動がおかしくなることがある。
buffer_comment = 5

# csvの保存先
now = datetime.datetime.now()
filename_csv  = f"/app/output/csv/{now.strftime('%Y%m%d_%H%M%S')}.csv"
filename_html = f"/app/output/html/{now.strftime('%Y%m%d_%H%M%S')}.html"

## 初期準備

In [3]:
# output用csvファイルを作成
with open(filename_csv, mode='a') as f:
    f.write("timestamp,image_url,name,amount")

In [4]:
# 現在時刻取得用
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')

## 関数定義

In [5]:
# 取得したコメントを整形するプログラム
def clean_comment(comments):

    # li のWebElementを引数とする

    text = ""
    dict_gift = {}

    # 名前の取得
    try:
        name = comments.find_element(By.XPATH,".//div[@class='comment-name']//p[@class='text-box']").text
        text += name + "：\n"

    except NoSuchElementException:
        # 例外処理
        pass

    # コメントの取得
    try:
        comment = comments.find_element(By.XPATH,".//div[contains(@class,'comment-text')]").text
        text += comment
    except NoSuchElementException:
        # 例外処理
        pass

    # ギフトの取得
    try:

        # text += f"{comments.text}"
        gift = comments.find_element(By.XPATH,"./button/div")
        gift_img  = gift.find_element(By.XPATH, ".//img").get_attribute("src")
        gift_name = gift.find_element(By.XPATH, "./div[2]").text
        gift_amount = gift.find_element(By.XPATH, "./pre").text

        text += f"{gift_img}\n"
        text += f"{gift_name}\n"
        text += f"{gift_amount}"

        dict_gift["timestamp"] = datetime.datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')
        dict_gift["image_url"] = gift_img
        dict_gift["name"]      = gift_name
        dict_gift["amount"]    = gift_amount

    except NoSuchElementException:
        # 例外処理
        pass
    return(text, dict_gift)

## Liveページ表示

In [6]:
# Chromeを起動
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1200,1000")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Remote(
    command_executor = os.environ["SELENIUM_URL"],
    options = options
)
driver.get(live_url)

# ページが読み込まれるまで待機()
time.sleep(5)

# リダイレクトを考慮してURLを更新
live_url = driver.current_url

# 再生ボタンをクリック
button_xpath = "//div[@class='live-detail-chat-container listener']/div[2]/button"
element = driver.find_element(By.XPATH, button_xpath)
element.click()

## ギフト取得
途中でギフト取得をやめる場合はコードの実行を中止してOK

In [7]:
pre_text = []

# コメントを監視するループ
while 1:

    # 直近 buffer_comment 個のコメントを取得
    recent_comments = driver.find_elements(By.XPATH, "//div[@class='native-scroll-bars']//li")[-buffer_comment:]

    recent_text = []
    recent_gift = []

    for comment in recent_comments:
        text, dict_gift = clean_comment(comment)
        recent_text.append(text)
        recent_gift.append(dict_gift)

    # コメントが更新しているかを確認
    if pre_text != recent_text:
        
        # 新しいコメントがいくつあるかをチェック
        if len(pre_text) != len(recent_text):
            num_new = len(recent_text) - len(pre_text)
            if num_new>buffer_comment:
                print("Warning:バッファーが溢れました")
        else:
            for num_new in range(0, buffer_comment) :
                if pre_text[num_new:] == recent_text[:-num_new]: break
            else:
                num_new = buffer_comment
                print("Warning:バッファーが溢れました")

        # 新しいコメントを表示
        for i in range(0,num_new):
            print(recent_text[-num_new+i])

            # ギフト情報があればcsvに書き込む
            dict_gift = recent_gift[-num_new+i]
            if any(dict_gift):
                with open(filename_csv, mode='a') as f:
                    f.write(f'\n{dict_gift["timestamp"]},{dict_gift["image_url"]},"{dict_gift["name"]}",{dict_gift["amount"]}')

        # 更新処理
        pre_text= recent_text

    # 枠が閉じていたらループを修了
    if driver.current_url != live_url: break

    # 待機
    time.sleep(wait_time)



⚪️しろくまパパ #本日21時生誕祭🎂：
生誕祭🎉🎂
尊いか、手ブラ自撮りか、
100文字以上200文字以内の愛のこもったお便り待ってるね！
初めてのカウントダウン枠にようこそ✨
日付け跨いだら5分て閉じます

入退室、無言、潜り、挨拶するし
はゆー桃巳(ﾄｳﾐ)：
ジップロック大事
⟬笹蔵 羽月希,⟭：
( ˙꒳˙  )oh......
じゃぁいつかDarlingからボイメ来たら私も画録来て音声に切り替えよ
して
wwwwwwwww
はゆー桃巳(ﾄｳﾐ)：
いいなぁ(´・ω・｀)
⟬笹蔵 羽月希,⟭：
(b｀>▽<´)-bｲｴｰｲ☆゛
はゆー桃巳(ﾄｳﾐ)：
ﾜｧ───ヽ(*ﾟ∀ﾟ*)ﾉ───ｲ
⟬笹蔵 羽月希,⟭：
待ってるー✌️
はゆー桃巳(ﾄｳﾐ)：
12月ですのでだいぶ遅いです
⟬笹蔵 羽月希,⟭：
1/19
はゆー桃巳(ﾄｳﾐ)：
12月8日
⟬笹蔵 羽月希,⟭：
ささくらな
˙˚ʚおそのɞ˚˙：
11月19日
ちか：
8/22
千樹 ✻⚪✻：
先輩と合流していくので落ちます！
パパさん、ほんとにおめでとう🎉
⟬笹蔵 羽月希,⟭：
千樹ちゃん行ってらっしゃい！
ちか：
千樹さんまたねー
˙˚ʚおそのɞ˚˙：
行ってらっしゃい
⟬笹蔵 羽月希,⟭：
忘れないでね😡
忘れてたらオコだよ٩(◦`^´◦)۶
彗#アすニ橋を翔ける：
Darling彗もボイメおくったから後で聞いといてー
彗もちょっとやることあるからおちちゃうけどごめんへ
˙˚ʚおそのɞ˚˙：
えびちゃん5/18
彗#アすニ橋を翔ける：
彗のはフリ素だからながしたいならごじゆーに
((ヾ( ˙꒳˙🐾ᵕ̈)ﾌﾘﾌﾘ
🍊ꪑ*蜜柑@ﾐｶﾝ：
えびちゃん、湧く待ってるからね♡
🦐🍊えび塩ゆず七味：
(*°ㅁ°)ﾊｯ‼‼
🍊ꪑ*蜜柑@ﾐｶﾝ：
シグマさん、やってるね
拗ねた(╺ ∧╺ )
⟬笹蔵 羽月希,⟭：
www
🦐🍊えび塩ゆず七味：
今日の枠遅れます
( ´ ꒳ ` )ﾉ
⟬笹蔵 羽月希,⟭：
ガオ( ´･ω･`)
🍑ᐝ杏萌momo：
かまちょ𝔻𝕒𝕣𝕝𝕚𝕟𝕘🤣
https://static.spooncast.net/jp/item-store/product-item/p/2ffff974-b51c-475f-b023-f3fc4f67184d_1715670601019.png
🦐🍊えび塩ゆず七

KeyboardInterrupt: 

In [24]:
# ブラウザを閉じる
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.41)
Stacktrace:
#0 0x55b70f492533 <unknown>
#1 0x55b70f1f107f <unknown>
#2 0x55b70f1df3f4 <unknown>
#3 0x55b70f1d0d2d <unknown>
#4 0x55b70f24c9eb <unknown>
#5 0x55b70f244f08 <unknown>
#6 0x55b70f21a71a <unknown>
#7 0x55b70f21b875 <unknown>
#8 0x55b70f4d6e1d <unknown>
#9 0x55b70f4da751 <unknown>
#10 0x55b70f4c107e <unknown>
#11 0x55b70f4db388 <unknown>
#12 0x55b70f4b5fe0 <unknown>
#13 0x55b70f4f7748 <unknown>
#14 0x55b70f4f78c8 <unknown>
#15 0x55b70f51170d <unknown>
#16 0x7f74363d6609 <unknown>


## まとめhtmlファイルを作成

In [8]:
# 作成したcsvファイルをpd.DataFrame形式で読み込み
df = pd.read_csv(filename_csv)

In [9]:
# Spoon一覧のためのhtmlを生成
contents = ""

for _, data in df.iterrows():
    contents += f"""
        <div class="spoon_div">
            <div class="image_div">
                <img src="{data['image_url']}" alt="{data['name'].strip('"')}" class="image">
            </div>
            <div class="user_div">
                <p class="name">{data['name'].strip('"')}</p>
            </div>
                <pre class="amount">{data['amount']}</pre>
            </div>
            <br><br>
    """

In [10]:
# 全体のコードを生成
code = f"""
<!DOCTYPE html>
<html lang="ja">
    <head>
        <meta charset="utf-8"> 
        <title>Spoonスクショ</title>
        <link rel="stylesheet" href="./style.css">
    </head>
    <body>
    {contents}
    </body>
</html>
"""

In [11]:
# htmlファイル書き込み
with open(filename_html, mode='w') as f:
    f.write(code)